In [ ]:
import geopandas

from carbonplan_buffer_analysis import utils
from carbonplan_buffer_analysis.prefect.flows.calculate_buffer_contributions import (
    get_issuance_table,
)
from carbonplan_buffer_analysis.prefect.tasks.project_reversals import (
    load_fires,
)
from carbonplan_forest_offsets.data import cat

M2_TO_ACRE = 4046.86

In [ ]:
retro_json = cat.project_db_json.read()

In [ ]:
fires = load_fires()

In [ ]:
geom = utils.load_project_geometry("CAR1046")
geom = geom.to_crs(fires.crs).buffer(0)

In [ ]:
route = fires.loc[fires["name"] == "ROUTE COMPLEX"]

In [ ]:
trinity_proj = [x for x in retro_json if x["opr_id"] == "CAR1046"][0]

In [ ]:
route_acres = (geopandas.clip(route, geom).area / M2_TO_ACRE).item()
f" Although it affected only about {route_acres/ trinity_proj['acreage'] * 100:.0f} percent of the project’s total acreage, the carbon losses were so large that a post-fire verification found onsite carbon had fallen below the project’s baseline scenario."

In [ ]:
ax = route.plot(color=light["red"])
geom.plot(ax=ax, color=".3")

## Credited vs onsite carbon

The text we want to populate is:

> To give quantitative context, we examine a subset of TK forest offset projects
> from a dataset assembled for an earlier research project (Badgley et al.,
> 2021). In the initial crediting period, these projects collectively earned TK
> gross credits, of which TK credits were contributed to the buffer pool.
> Although this implies a buffer pool contribution of TK percent of the total
> credit gross carbon, the onsite carbon stocks at these projects contained TK
> tCO₂. From the perspective of onside carbon that is subject to permanence
> risks, the buffer pool contribution for these projects is only TK percent.


In [ ]:
n_subset = len(retro_json)
f"we examine a subset of {n_subset} forest offset projects from a dataset assembled for"

In [ ]:
initial_buffer = sum([x["buffer_contribution"] for x in retro_json])
initial_credits = sum([x["arbocs"]["issuance"] for x in retro_json])
initial_onsite = sum(
    [x["rp_1"]["ifm_1"] + x["rp_1"]["ifm_3"] for x in retro_json]
)

In [ ]:
f"In the initial crediting period, these projects collectively earned {initial_credits / 1_000_000:.1f} gross credits, of which {initial_buffer / 1_000_000:.1f} million credits were contributed to the buffer pool."

In [ ]:
f"Although this implies a buffer pool contribution of {initial_buffer/initial_credits * 100:.2f} percent of the total credit gross carbon, the onsite carbon stocks at these projects contained {initial_onsite / 1_000_000:.1f} tCO₂."

In [ ]:
f"From the perspective of onside carbon that is subject to permanence risks, the buffer pool contribution for these projects is only {initial_buffer / initial_onsite * 100:.2f} percent. "

# Conclusion


In [ ]:
issuance = get_issuance_table.run()

gross_buffer = issuance.buffer_pool.sum()
gross_allocation = issuance.allocation.sum()

In [ ]:
f"As of our study cut-off date of January 5, 2022, a total of 31.0 million credits ({gross_buffer/gross_allocation * 100:.1f} percent) had been set aside in the buffer pool to insure an additional portfolio of {(gross_allocation - gross_buffer) / 1_000_000:.1f} million credits that are sold in private carbon markets and used to comply with California’s cap-and-trade program."